In [1]:
# Perform required imports
import json
import psycopg2
import logging

In [2]:
# Read the json file
with open('purchase_orders.json', 'r') as json_file:
    json_data = json.load(json_file)
    print(json_data)

{'PurchaseOrders': [{'PurchaseOrderNo': 1, 'Address': [{'Type': 'Shipping', 'Name': 'Ellen Adams', 'Street': '123 Maple Street', 'City': 'Mill Valley', 'State': 'CA', 'Zip': 10999, 'Country': 'USA'}, {'Type': 'Billing', 'Name': 'Tai Yee', 'Street': '8 Oak Avenue', 'City': 'Old Town', 'State': 'PA', 'Zip': 95819, 'Country': 'USA'}], 'DeliveryNotes': 'Please leave packages in shed by driveway.', 'Items': {'Item': [{'PartNumber': '872-AA', 'ProductName': 'Lawnmower', 'Quantity': 1, 'USPrice': 148.95, 'Comment': 'Confirm this is electric'}, {'PartNumber': '926-AA', 'ProductName': 'Baby Monitor', 'Quantity': 2, 'USPrice': 39.98, 'ShipDate': '1999-05-21'}]}}, {'PurchaseOrderNo': 2, 'Address': [{'Type': 'Shipping', 'Name': 'Cristian Osorio', 'Street': '456 Main Street', 'City': 'Buffalo', 'State': 'NY', 'Zip': 98112, 'Country': 'USA'}, {'Type': 'Billing', 'Name': 'Cristian Osorio', 'Street': '456 Main Street', 'City': 'Buffalo', 'State': 'NY', 'Zip': 98112, 'Country': 'USA'}], 'DeliveryNotes'

In [3]:
# Define table columns
columns = [
    "po",
    "billing_name",
    "billing_street",
    "billing_city",
    "billing_state",
    "billing_zip",
    "billing_country",
    "items"
]

In [4]:
# Define a connect_to_database function
def connect_to_database():
    db_conn = {
        'user': 'postgres',
        'password': 'postgres',
        'database': 'postgres',
        'host': 'localhost',
        'port': 15432
    }
    try:
        conn = psycopg2.connect(**db_conn)
        return conn
    except Exception as e:
        logging.error("Error: Unable to connect to the database")
        logging.error(e)
        return None

In [5]:
# Instantiate a database connection and creates a table
conn = connect_to_database()

cur = conn.cursor()

create_sql = """
DROP TABLE IF EXISTS purchase_orders;
CREATE TABLE purchase_orders (
    po INT PRIMARY KEY,
    billing_name TEXT,
    billing_street TEXT,
    billing_city TEXT,
    billing_state TEXT,
    billing_zip INT,
    billing_country TEXT,
    items TEXT
)
"""
with cur as cursor:
    cur.execute(create_sql)

conn.commit()

In [6]:
# Reads the JSON and insert into purchase_orders table
conn = connect_to_database()

cur = conn.cursor()

for product in json_data["PurchaseOrders"]:
    po = product['PurchaseOrderNo']
    billing_address = product['Address'][1]
    billing_name = billing_address['Name']
    billing_street = billing_address['Street']
    billing_city = billing_address['City']
    billing_state = billing_address['State']
    billing_zip = billing_address['Zip']
    billing_country = billing_address['Country']

    items = product.get('Items', {}).get('Item', [])
    
    if isinstance(items, list):
        formatted_items = " | ".join([
            f"{item['PartNumber']}:{item['Quantity']}:{item['USPrice']}" for item in items
        ])
    elif isinstance(items, dict):
        formatted_items = f"{items['PartNumber']}:{items['Quantity']}:{items['USPrice']}"

    insert_sql = """
        INSERT INTO purchase_orders ({columns})
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
    """.format(columns=", ".join(columns))

    values = (po, billing_name, billing_street, billing_city, billing_state, billing_zip, billing_country, formatted_items)

    cur.execute(insert_sql, values)

In [7]:
# Query the inserted data
sql_query = "SELECT * FROM purchase_orders;"
cur = conn.cursor()
cur.execute(sql_query)
rows = cur.fetchall()
for row in rows:
    print(row)

(1, 'Tai Yee', '8 Oak Avenue', 'Old Town', 'PA', 95819, 'USA', '872-AA:1:148.95 | 926-AA:2:39.98')
(2, 'Cristian Osorio', '456 Main Street', 'Buffalo', 'NY', 98112, 'USA', '456-NM:1:45.99')
(3, 'Jessica Arnold', '4055 Madison Ave', 'Buffalo', 'NY', 98112, 'USA', '898-AZ:1:29.99 | 898-AM:1:14.99')


In [8]:
# Closes the cursor and the connection
cur.close()
conn.close()